In [ ]:
import numpy as np
from biom.table import Table
from biom import load_table
import pandas as pd

In [ ]:
# t = load_table("/Volumes/TBHD/Valencia/Microbiome_Analysis/Nephele_cloud_play_project/outputs/for_analyze_with_microbiomedb.biom")
# t = load_table("/Volumes/TBHD_share/valencia/3sample/no_merge_trim+trim_output/for_analyze_with_microbiomedb.biom")
# t = load_table("/Volumes/TBHD_share/valencia/3sample/batch2/wgsa2/outputs/for_analyze_with_microbiomedb.biom")
# t = load_table("/Volumes/TBHD_share/valencia/3sample/batch2/wgsa2/outputs/TAXprofiles/MAGs_TAX/MAG-based_Counts+TAX.biom")
# t = load_table("/Volumes/TBHD_share/walitt_sample/3sample/outputs/for_analyze_with_microbiomedb.biom")
t = load_table("/Volumes/TBHD_share/valencia/bmock12/NEPHELE/wgsa2/subset_bmock12/outputs/for_analyze_with_microbiomedb.biom")
print(t)
t.ids(axis='observation')

In [ ]:
# Use 6 for species, 5 for genus.

phylum_idx = 5
# Somtimes taxonomy is capitalized, sometimes not.
collapse_f = lambda id_, md: '; '.join(md['Taxonomy'][phylum_idx:phylum_idx + 1])
collapsed = t.collapse(collapse_f, axis='observation')

df = collapsed.to_dataframe()
display(df)

col1 = pd.DataFrame(df.iloc[:, :])
display(col1)

col1 = col1.sparse.to_dense()
col1.rename(index={'':'Unclassified'},inplace=True)

print(col1.sum(axis=0))
# col1.to_csv('test.csv')
# col1.drop('Unclassified_sp', inplace=True)

In [ ]:
# Percent abundance operation.
pct = col1.apply(lambda x: x / x.sum(), axis=0)
display(pct)

# Drop features with less than x percent abundance.
pct = pct.where(pct >= 0.001).dropna()
# pct.to_csv("pipelines/bmock12/wgsa2/fullpct.csv")

# Add other category to account for dropped samples.
pct.loc['Rare Taxa']= 1.0 - pct.sum(numeric_only=True, axis=0)
display(pct)
# pct.to_csv("pipelines/bmock12/wgsa2/0-001pct.csv")

# import matplotlib
# matplotlib.style.use('ggplot') 
pct.T.plot.bar(stacked=True, figsize=(10, 10), ylabel="Fraction", xlabel="Sample", title="Alignment Genus").legend(loc='center left', bbox_to_anchor=(1.0, 0.5), title="Genus")

In [ ]:
# Sort the values by the first sample in ascending order.
pct.T
pct.sort_values(axis=0, ascending=False, by=pct.columns[0], inplace=True)

In [ ]:
# Plot with seaborn.
import plotly.express as px
fig = px.bar(pct.T, x=pct.index, y=pct.columns, title="Alignment Species", labels={'index': 'Sample', 'value': 'Fraction', "variable": "Species"})
fig.show()

In [ ]:
# Sanity check to make sure my operations are correct.

test_df = pd.DataFrame([[1, 2, 3], [4, 5, 6], [7, 8, 9]], columns=['a', 'b', 'c'])

pct_test = test_df.apply(lambda x: x / x.sum(), axis=0)
display(pct_test)

pct_test = pct_test.where(pct_test >= 0.50).dropna()
display(pct_test)

pct_test.loc['Column_Total']= 1.0 - pct_test.sum(numeric_only=True, axis=0)
display(pct_test)